We will show the main RePlay functionality and compare performance of RePlay models on well-known MovieLens dataset. For simplicity we consider here only the variou bandit algorithms with context. The list of considered strategies for comparison:

1) Linear UCB

2) Linear Thompson Sampling

3) Logistic Thompson Sampling

### Dataset

We will compare RePlay models on MovieLens 1m.

### Dataset preprocessing:

Ratings greater than or equal to 3 are considered as positive interactions.

### Data split

Dataset is split by date so that 20% of the last interactions as are placed in the test part. Cold items and users are dropped.

### Predict:
We will predict top-10 most relevant films for each user.

### Metrics

Quality metrics used: ndcg@k, hitrate@k, map@k, mrr@k for k = 1, 5, 10 Additional metrics used: coverage@k and surprisal@k.


In [ ]:
! pip install rs-datasets

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import warnings
from optuna.exceptions import ExperimentalWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ExperimentalWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import logging
import time
import pandas as pd
import numpy as np

from pyspark.sql import functions as sf, types as st
from pyspark.sql.types import IntegerType

from replay.experimental.preprocessing.data_preparator import Indexer, DataPreparator
from replay.metrics import Coverage, HitRate, MRR, MAP, NDCG, Surprisal, Experiment
from replay.experimental.models import (
    NeuralTS
)


from replay.utils.session_handler import State
from replay.splitters import TimeSplitter
from replay.utils.spark_utils import get_log_info
from rs_datasets import MovieLens

In [ ]:
spark = State().session
spark

In [ ]:
spark.sparkContext.setLogLevel('ERROR')

In [ ]:
logger = logging.getLogger("replay")

In [ ]:
K = 10
K_list_metrics = [1, 5, 10]
BUDGET = 20
BUDGET_NN = 10
SEED = 12345

## Preprocessing 

### Data loading

In [ ]:
data = MovieLens("1m")
data.info()

In [ ]:
data.ratings

In [ ]:
preparator = DataPreparator()

In [ ]:
%%time
log = preparator.transform(columns_mapping={'user_id': 'user_id',
                                      'item_id': 'item_id',
                                      'relevance': 'rating',
                                      'timestamp': 'timestamp'
                                     }, 
                           data=data.ratings)

In [ ]:
# will consider ratings >= 3 as positive feedback. A positive feedback is treated with relevance = 1
only_positives_log = log.filter(sf.col('relevance') >= 3).withColumn('relevance', sf.lit(1))
only_positives_log.count()

### Indexing 

In [ ]:
indexer = Indexer(user_col='user_id', item_col='item_id')

In [ ]:
%%time
indexer.fit(users=log.select('user_id'),
           items=log.select('item_id'))

In [ ]:
%%time
log_replay = indexer.transform(df=only_positives_log)
log_replay.show(2)
log_replay.count()

### Data split

In [ ]:
# train/test split 
train_spl = TimeSplitter(
    time_threshold=0.2,
    drop_cold_items=True,
    drop_cold_users=True,
    query_column="user_idx",
    item_column="item_idx",
)

train, test = train_spl.split(log_replay)
print('train info:\n', get_log_info(train))
print('test info:\n', get_log_info(test))

In [ ]:
train.is_cached

In [ ]:
# train/test split for hyperparameters selection
opt_train, opt_val = train_spl.split(train)
opt_train.count(), opt_val.count()

In [ ]:
opt_train.is_cached

In [ ]:
# negative feedback will be used for Wilson and UCB models
only_negatives_log = indexer.transform(df=log.filter(sf.col('relevance') < 3).withColumn('relevance', sf.lit(0.)))
test_start = test.agg(sf.min('timestamp')).collect()[0][0]

# train with both positive and negative feedback
pos_neg_train=(train
              .withColumn('relevance', sf.lit(1.))
              .union(only_negatives_log.filter(sf.col('timestamp') < test_start))
             )
pos_neg_train.cache()
pos_neg_train.count()

In [ ]:
pos_neg_train.is_cached

In [ ]:
pos_neg_train.groupBy('relevance').count().show()

### Item features 

In [ ]:
item_features_original = preparator.transform(columns_mapping={'item_id': 'item_id'}, 
                           data=data.items)
item_features = indexer.transform(df=item_features_original)
item_features.show(2)
#different item features

from pyspark.sql.functions import max,min
item_features.select(max(item_features.item_idx)).show()
item_features.select(min(item_features.item_idx)).show()
#just to check that the indexing is dense between 0 and 3882
item_features.count()

In [ ]:
year = item_features.withColumn('year', sf.substring(sf.col('title'), -5, 4).astype(st.IntegerType())).select('item_idx', 'year')
year.show(2)

In [ ]:
genres = (
    item_features.select(
        "item_idx",
        sf.split("genres", "\|").alias("genres")
    )
)

In [ ]:
genres_list = (
    genres.select(sf.explode("genres").alias("genre"))
    .distinct().filter('genre <> "(no genres listed)"')
    .toPandas()["genre"].tolist()
)

In [ ]:
item_features = genres
for genre in genres_list:
    item_features = item_features.withColumn(
        genre,
        sf.array_contains(sf.col("genres"), genre).astype(IntegerType())
    )
item_features = item_features.drop("genres").cache()
item_features.count()
item_features = item_features.join(year, on='item_idx', how='inner')

In [ ]:
item_features = item_features.withColumnRenamed("Children's","Children")
item_features = item_features.toPandas()

In [ ]:
item_features.columns

### Users features

In [ ]:
data.users.head()

In [ ]:
#same preprocessing for users as was done in 2.4.1.
user_features_original = preparator.transform(columns_mapping={'user_id': 'user_id'}, 
                           data=data.users)
user_features = indexer.transform(df=user_features_original)
#switch for a while into pandas
user_features = user_features.toPandas()
user_features.head(2)

In [ ]:
from pyspark.sql.functions import mean, stddev
user_train_mean = pos_neg_train.groupBy('user_idx').mean('relevance').withColumnRenamed('avg(relevance)','user_mean_rel').toPandas()
user_train_count = pos_neg_train.groupBy('user_idx').count().withColumnRenamed('count','user_count_rel').toPandas()
user_train_std = pos_neg_train.groupBy('user_idx').agg({'relevance': 'stddev'}).withColumnRenamed('stddev(relevance)','user_std_rel').toPandas()
user_train_median = pd.DataFrame(pos_neg_train.toPandas().groupby('user_idx', as_index = False)['relevance'].agg('median')).rename(columns={'relevance': 'user_median_rel'})
user_train_kurt = pd.DataFrame(pos_neg_train.toPandas().groupby('user_idx', as_index = False)['relevance'].apply(pd.DataFrame.kurt)).rename(columns={'relevance': 'user_kurt_rel'})
user_train_skew = pd.DataFrame(pos_neg_train.toPandas().groupby('user_idx', as_index = False)['relevance'].apply(pd.DataFrame.skew)).rename(columns={'relevance': 'user_skew_rel'})

item_train_mean = pos_neg_train.groupBy('item_idx').mean('relevance').withColumnRenamed('avg(relevance)','item_mean_rel').toPandas()
item_train_count = pos_neg_train.groupBy('item_idx').count().withColumnRenamed('count','item_count_rel').toPandas()
item_train_std = pos_neg_train.groupBy('item_idx').agg({'relevance': 'stddev'}).withColumnRenamed('stddev(relevance)','item_std_rel').toPandas()
item_train_median = pd.DataFrame(pos_neg_train.toPandas().groupby('item_idx', as_index = False)['relevance'].agg('median')).rename(columns={'relevance': 'item_median_rel'})
item_train_kurt = pd.DataFrame(pos_neg_train.toPandas().groupby('item_idx', as_index = False)['relevance'].apply(pd.DataFrame.kurt)).rename(columns={'relevance': 'item_kurt_rel'})
item_train_skew = pd.DataFrame(pos_neg_train.toPandas().groupby('item_idx', as_index = False)['relevance'].apply(pd.DataFrame.skew)).rename(columns={'relevance': 'item_skew_rel'})

full_mean = pos_neg_train.select(mean('relevance')).toPandas().values.squeeze()
full_count = 0
full_std = pos_neg_train.agg({'relevance': 'stddev'}).toPandas().values.squeeze()
full_median = pos_neg_train.toPandas()['relevance'].median().squeeze()
full_kurt = pos_neg_train.toPandas()['relevance'].kurt().squeeze()
full_skew = pos_neg_train.toPandas()['relevance'].skew().squeeze()

In [ ]:
user_train_sum = pos_neg_train.groupBy('user_idx').sum('relevance').withColumnRenamed('sum(relevance)','user_sum_rel').toPandas()
item_train_sum = pos_neg_train.groupBy('item_idx').sum('relevance').withColumnRenamed('sum(relevance)','item_sum_rel').toPandas()

In [ ]:
item_train_sum.head(10)

In [ ]:
user_all_true = pd.DataFrame({'user_idx': user_train_sum['user_idx'].values , 
                              'user_all_true': (user_train_sum['user_sum_rel'] == user_train_count['user_count_rel'].values).astype(int)})
item_all_true = pd.DataFrame({'item_idx': item_train_sum['item_idx'].values , 
                              'item_all_true': (item_train_sum['item_sum_rel'] == item_train_count['item_count_rel'].values).astype(int)})

In [ ]:
pop_item = pd.DataFrame({'item_idx': item_train_mean['item_idx'].values , 
                              'item_is_pop': (item_train_mean['item_mean_rel'] > 0.9).astype(int)})

In [ ]:
from sklearn.preprocessing import OneHotEncoder
print("max ocupation index: ", user_features['occupation'].max())
print("min ocupation index: ", user_features['occupation'].min())
count_diff_zips = user_features['zip_code'].unique().size
print("different zip codes: ", count_diff_zips) 
users_pd = user_features
users_pd['zip_code'] = users_pd['zip_code'].apply(lambda x: x[0])

#binarize age variable
bins = [0, 20, 30, 40, 50, 60, np.inf]
names = ['<20', '20-29', '30-39','40-49', '51-60', '60+']

users_pd['agegroup'] = pd.cut(users_pd['age'], bins, labels=names)
# users_pd = users_pd.drop(["age"], axis = 1)
# users_pd.head()

# #binarize following https://github.com/kfoofw/bandit_simulations/tree/master
# columnsToEncode = ["agegroup","gender","occupation"]
# myEncoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
# myEncoder.fit(users_pd[columnsToEncode])

# users_pd = pd.concat([users_pd.drop(columnsToEncode, 1),
#                            pd.DataFrame(myEncoder.transform(users_pd[columnsToEncode]), 
#                                         columns = myEncoder.get_feature_names_out(columnsToEncode))], axis=1).reindex()
users_pd['gender_occupation'] = users_pd['gender'] + '_' + users_pd['occupation'].astype(str)
users_pd.head()

In [ ]:
#stats
users_pd = users_pd.merge(user_train_mean, on='user_idx', how='left')
users_pd['user_mean_rel'].fillna(full_mean, inplace = True)

users_pd = users_pd.merge(user_train_count, on='user_idx', how='left')
users_pd['user_count_rel'].fillna(full_count, inplace = True)

users_pd = users_pd.merge(user_train_std, on='user_idx', how='left')
users_pd['user_std_rel'].fillna(full_std, inplace = True)

users_pd = users_pd.merge(user_train_median, on='user_idx', how='left')
users_pd['user_median_rel'].fillna(full_median, inplace = True)

users_pd = users_pd.merge(user_train_kurt, on='user_idx', how='left')
users_pd['user_kurt_rel'].fillna(full_kurt, inplace = True)

users_pd = users_pd.merge(user_train_skew, on='user_idx', how='left')
users_pd['user_skew_rel'].fillna(full_skew, inplace = True)

users_pd = users_pd.merge(user_all_true, on='user_idx', how='left')
users_pd['user_all_true'].fillna(0, inplace = True)


item_features = item_features.merge(item_train_mean, on='item_idx', how='left')
item_features['item_mean_rel'].fillna(full_mean, inplace = True)

item_features = item_features.merge(item_train_count, on='item_idx', how='left')
item_features['item_count_rel'].fillna(full_count, inplace = True)

item_features = item_features.merge(item_train_std, on='item_idx', how='left')
item_features['item_std_rel'].fillna(full_std, inplace = True)

item_features = item_features.merge(item_train_median, on='item_idx', how='left')
item_features['item_median_rel'].fillna(full_median, inplace = True)

item_features = item_features.merge(item_train_kurt, on='item_idx', how='left')
item_features['item_kurt_rel'].fillna(full_kurt, inplace = True)

item_features = item_features.merge(item_train_skew, on='item_idx', how='left')
item_features['item_skew_rel'].fillna(full_skew, inplace = True)

item_features = item_features.merge(item_all_true, on='item_idx', how='left')
item_features['item_all_true'].fillna(0, inplace = True)

item_features = item_features.merge(pop_item, on='item_idx', how='left')
item_features['item_is_pop'].fillna(0, inplace = True)


In [ ]:
#make it pyspark
user_features = spark.createDataFrame(users_pd) 
user_features.cache()
user_features.printSchema()
print("total users: ",user_features.count())

In [ ]:
item_features = spark.createDataFrame(item_features) 
item_features.cache()
item_features.printSchema()
print("total users: ",item_features.count())

In [ ]:
# item_features = indexer.transform(df=item_features)
# user_features = indexer.transform(df=user_features)

In [ ]:
cat_embed_cols = ["agegroup", 'gender', 'occupation', "zip_code"]
continuous_cols = ["Crime", "Sci-Fi", "Musical", 
                   "Mystery", "Documentary", "Fantasy", "Children", 
                   "Drama", "Horror", "Adventure", "Western",
                   "Romance", "War", "Animation", "Action", "Comedy", "Thriller",
                   "Film-Noir"]
                  #  , "age",
                  #  'user_mean_rel', 'user_count_rel', 'user_std_rel',
                  #  'user_median_rel', 'user_kurt_rel', 'user_skew_rel',
                  # 'item_mean_rel', 'item_count_rel', 'item_std_rel',
                  #  'item_median_rel', 'item_kurt_rel', 'item_skew_rel']
wide_cols = ['gender', 'occupation']
crossed_cols = ['gender_occupation']

cols_item = {'continuous_cols':[], 'cat_embed_cols':[], 'wide_cols': []}

cols_user = {'continuous_cols':[], 'cat_embed_cols':[], 'wide_cols': []}

In [ ]:
e = Experiment(
    [ MAP(K), 
      NDCG(K), 
      HitRate(K_list_metrics), 
      Coverage(K),
      Surprisal(K),
      MRR(K)],
    test,
    pos_neg_train,
    query_column="user_idx", item_column="item_idx", rating_column="relevance"
)

In [ ]:
 bandit_models = {
         'Neural TS' : [NeuralTS(user_cols = cols_user,
                                    item_cols = cols_item,
                                    dim_head=20,
                                    deep_out_dim=20,
                                    hidden_layers=[32, 20],
                                    embedding_sizes=[32, 32, 64],
                                    wide_out_dim=1,
                                    head_dropout=0.8,
                                    deep_dropout=0.4,
                                    n_epochs=2,
                                    opt_lr = 3e-4,
                                    lr_min = 1e-5,
                                    use_gpu = False,
                                    plot_dir='test.png',
                                    use_warp_loss=True,
                                    cnt_neg_samples=200,
                                    cnt_samples_for_predict=10,
                                    exploration_coef = +1.0
                                ), 'no_opt']   
         }

In [ ]:
def fit_predict_add_res(name, model, experiment, train, test, suffix=''):
    """
    Run fit_predict for the `model`, measure time on fit_predict and evaluate metrics
    """
    start_time=time.time()
    
    logs = {'log': train}
    predict_params = {'k': K, 'users': test.select('user_idx').distinct()}
    
    if isinstance(model, (NeuralTS)):
        logs['log'] = pos_neg_train

    if isinstance(model, (NeuralTS)):
        logs['item_features'] = item_features
        logs['user_features'] = user_features 
    predict_params.update(logs)

    model.fit(**logs)
    fit_time = time.time() - start_time
    
    pred=model.predict(**predict_params)
    pred.show(100)
    pred.cache()
    pred.count()
    predict_time = time.time() - start_time - fit_time

    experiment.add_result(name + suffix, pred)
    metric_time = time.time() - start_time - fit_time - predict_time
    experiment.results.loc[name + suffix, 'fit_time'] = fit_time
    experiment.results.loc[name + suffix, 'predict_time'] = predict_time
    experiment.results.loc[name + suffix, 'metric_time'] = metric_time
    experiment.results.loc[name + suffix, 'full_time'] = (fit_time + 
                                                          predict_time +
                                                          metric_time)
    pred.unpersist()
    print(experiment.results[['NDCG@{}'.format(K), 'MRR@{}'.format(K), 'Coverage@{}'.format(K), 'fit_time']].sort_values('NDCG@{}'.format(K), ascending=False))
    #add for me
    return pred

In [ ]:
def full_pipeline(models, experiment, train, test, suffix='', budget=BUDGET):
    """
    For each model:
        -  if required: run hyperparameters search, set best params and save param values to `experiment`
        - pass model to `fit_predict_add_res`        
    """
    
    for name, [model, params] in models.items():
        model.logger.info(msg='{} started'.format(name))
        if params != 'no_opt':
            model.logger.info(msg='{} optimization started'.format(name))
            best_params = model.optimize(opt_train, 
                                         opt_val, 
                                         param_borders=params, 
                                         item_features=item_features,
                                         user_features=user_features,
                                         k=K, 
                                         budget=budget)
            logger.info(msg='best params for {} are: {}'.format(name, best_params))
            model.set_params(**best_params)
        
        logger.info(msg='{} fit_predict started'.format(name))
        pred = fit_predict_add_res(name, model, experiment, train, test, suffix)
        # here we call protected attribute to get all parameters set during model initialization
        experiment.results.loc[name + suffix, 'params'] = str(model._init_args)
        #add for me
        return pred

In [ ]:
%%time
pred = full_pipeline(bandit_models, e, train, test)

In [ ]:
e.results.sort_values('NDCG@10', ascending=False)